# Hedge Fund Positioning

The Federal Reserve signalled a major shift in monetary policy stance at the end of 2021. How has hedge fund positioning changed in a rising rates environment?
We start by fetching global positioning data from January 2020 from our Prime Book Flows and Exposures (PBFA) dataset.

In [ ]:
# Import libraries
import datetime
from gs_quant.data import Dataset
from gs_quant.session import GsSession, Environment
import matplotlib.pyplot as plt

In [ ]:
# Graphical formatting
import matplotlib.dates as mdates
import matplotlib.ticker as mtick
half_year_locator = mdates.MonthLocator(interval=6)
year_month_formatter = mdates.DateFormatter("%Y-%m") # four digits for year, two for month
pct_formatter = mtick.PercentFormatter(xmax=1,decimals=0)

In [ ]:
# Authentication
client_id = # <insert client id here>
client_secret = # <insert client secret here>
GsSession.use(Environment.PROD, client_id, client_secret, ('read_product_data',))

In [ ]:
# Fetch positioning data from January 2020
ds = Dataset('PBFA')
data = ds.get_data(datetime.date(2020, 1, 1))
data = data.reset_index()
data.date = data.date.apply(lambda x: x.date())

We can then focus on sector rotation. We expect rising rates to have a significant impact on growth companies because their valuation is sensitive to the discount rate applied to earnings expected in the distant future.
Sector positioning is available by selecting the appropriate GICS level 1 sectors and displaying the values in the “clientNetWeight” column.

In [ ]:
# Plot weight of Technology, Energy and Financials sectors
glob_pos = data.loc[(data.clientStrategy=="All") & (data.region=="Global") & \
                    (data.sectorGICS1.isin(['Information Technology','Financials','Energy'])) ]
glob_pos = glob_pos.pivot(index="date", columns="sectorGICS1", values="clientNetWeight")

fig, ax = plt.subplots(figsize=(10, 6))
ax.plot(glob_pos.index, glob_pos['Financials'], 'g-',label='Financials')
ax.plot(glob_pos.index, glob_pos['Information Technology'], 'r-',label='Technology')
ax.plot(glob_pos.index, glob_pos['Energy'], 'b-',label='Energy')
ax.set_title('Global Hedge Fund Positioning')
ax.set_xlabel('Time')
ax.set_ylabel('Prime book weight', color='black')
ax.xaxis.set_major_locator(half_year_locator)
ax.xaxis.set_major_formatter(year_month_formatter)
ax.yaxis.set_major_formatter(pct_formatter)
fig.autofmt_xdate()
ax.legend(loc='upper right')

The results suggest that global hedge funds rotated out of Technology stocks from the end of 2021, bringing the sector weight down from 22% to 15%.
The rotation benefitted Financials, which briefly overtook Technology in terms of net weight at the end of 2022, and Energy. Thelatter was driven by soaring energy prices, particularly as the conflict in Ukraine started in February 2022.
The beginning of 2023 saw a partial reversal of this trend, arguably due to cooling inflation expectations and a general improvement in the global macroeconomic outlook.

Is the pattern driven by any region in particular?
The previous chart considered positions in global equities but we can restrict the analysis to a specific region, e.g. Europe, i.e. focus on equities issued by European firms.
Here we rescale sector weights by the total net weight allocated to Europe in the global PB book so that sector weights sum up to one and are comparable to those displayed in the previous graph.

In [ ]:
# Plot weight of Technology, Energy and Financials sectors in Europe
eur_pos = data.loc[(data.clientStrategy=="All") & (data.region=="Europe") & \
                    (data.sectorGICS1.isin(['Information Technology','Financials','Energy','All'])) ]
eur_pos = eur_pos.pivot(index="date", columns="sectorGICS1", values="clientNetWeight")

fig, ax = plt.subplots(figsize=(10, 6))
ax.plot(eur_pos.index, eur_pos['Financials']/eur_pos['All'], 'g-',label='Financials')
ax.plot(eur_pos.index, eur_pos['Information Technology']/eur_pos['All'], 'r-',label='Technology')
ax.plot(eur_pos.index, eur_pos['Energy']/eur_pos['All'], 'b-',label='Energy')
ax.set_title('Hedge Fund Positioning - European Stocks')
ax.set_xlabel('Time')
ax.set_ylabel('Prime book weight', color='black')
ax.xaxis.set_major_locator(half_year_locator)
ax.xaxis.set_major_formatter(year_month_formatter)
ax.yaxis.set_major_formatter(pct_formatter)
fig.autofmt_xdate()
ax.legend(loc='lower right')

The chart shows that the weight of Technology declined in Europe between the end of 2021 and the end of 2022, in line with our global results.
Exposure to Energy also displays a marked increase, particularly from February 2022.
Sector allocation is not as concentrated as in the global chart, where Technology exceeds 20% of the total net weight.

So far we have looked at portfolio weights, which can be affected by differences in return across sectors even if managers keep their holdings unchanged. For example, sustained outperformance of Energy stocks tends to boost the weight of the Energy sector in the portfolio.
Our datasets contain information on trading flows, split into long flows (long buying and long selling) and short flows (short covering and short selling).
Flows are converted into a z-score, which gives an indication of the intensity compared to history.
We extract flows for the Technology sector from January 2020 and display the data using a daily bar plot.

In [ ]:
# Plot long and short flow indicators for global Technology stocks
glob_long_flow = data.loc[(data.clientStrategy=="All") & (data.region=="Global") & \
                    (data.sectorGICS1.isin(['Information Technology','Financials','Energy'])) ]
glob_long_flow = glob_long_flow.pivot(index="date", columns="sectorGICS1", values="longLevel")
glob_short_flow = data.loc[(data.clientStrategy=="All") & (data.region=="Global") & \
                    (data.sectorGICS1.isin(['Information Technology','Financials','Energy'])) ]
glob_short_flow = glob_short_flow.pivot(index="date", columns="sectorGICS1", values="shortLevel")

fig, (ax1,ax2) = plt.subplots(2,1,figsize=(10, 6),sharey=True)
ax1.bar(glob_long_flow.index,glob_long_flow['Information Technology'],color="blue",width=4)
ax1.set_title('Long Flows - Technology')
ax1.set_ylabel('Z-score', color='black')
ax2.bar(glob_short_flow.index,glob_short_flow['Information Technology'],color="red",width=4)
ax2.set_ylabel('Z-score', color='black')
ax2.set_title('Short flows - Technology')
fig.autofmt_xdate()

Short flows throughout 2021 and 2022 appear strongly tilted towards short selling, while long flows are more balanced.
The short squeeze of January 2021 is evident in the chart, with risk-off flows (both long selling and short covering) that exceed 5 standard deviations compared to history.
In early 2023 we detect a sharp reversal in the directionality of the flows, with strong short covering in Technology stocks andmuted long flows.

## Disclaimers:
Indicative Terms/Pricing Levels: This material may contain indicative terms only, including but not limited to pricing levels. There is no representation that any transaction can or could have been effected at such terms or prices. Proposed terms and conditions are for discussion purposes only. Finalized terms and conditions are subject to further discussion and negotiation.
www.goldmansachs.com/disclaimer/sales-and-trading-invest-rec-disclosures.html If you are not accessing this material via Marquee ContentStream, a list of the author's investment recommendations disseminated during the preceding 12 months and the proportion of the author's recommendations that are 'buy', 'hold', 'sell' or other over the previous 12 months is available by logging into Marquee ContentStream using the link below. Alternatively, if you do not have access to Marquee ContentStream, please contact your usual GS representative who will be able to provide this information to you.
Please refer to https://marquee.gs.com/studio/ for price information of corporate equity securities.
Notice to Australian Investors: When this document is disseminated in Australia by Goldman Sachs & Co. LLC ("GSCO"), Goldman Sachs International ("GSI"), Goldman Sachs Bank Europe SE ("GSBE"), Goldman Sachs (Asia) L.L.C. ("GSALLC"), or Goldman Sachs (Singapore) Pte ("GSSP") (collectively the "GS entities"), this document, and any access to it, is intended only for a person that has first satisfied the GS entities that:
• the person is a Sophisticated or Professional Investor for the purposes of section 708 of the Corporations Act of Australia; and
• the person is a wholesale client for the purpose of section 761G of the Corporations Act of Australia.
To the extent that the GS entities are providing a financial service in Australia, the GS entities are each exempt from the requirement to hold an Australian financial services licence for the financial services they provide in Australia. Each of the GS entities are regulated by a foreign regulator under foreign laws which differ from Australian laws, specifically:
• GSCO is regulated by the US Securities and Exchange Commission under US laws;
• GSI is authorised by the Prudential Regulation Authority and regulated by the Financial Conduct Authority and the Prudential Regulation Authority, under UK laws;
• GSBE is subject to direct prudential supervision by the European Central Bank and in other respects is supervised by the German Federal Financial Supervisory Authority (Bundesanstalt für Finanzdienstleistungsaufischt, BaFin) and Deutsche Bundesbank;
• GSALLC is regulated by the Hong Kong Securities and Futures Commission under Hong Kong laws; and
• GSSP is regulated by the Monetary Authority of Singapore under Singapore laws.
Notice to Brazilian Investors
Marquee is not meant for the general public in Brazil. The services or products provided by or through Marquee, at any time, may not be offered or sold to the general public in Brazil. You have received a password granting access to Marquee exclusively due to your existing relationship with a GS business located in Brazil. The selection and engagement with any of the offered services or products through Marquee, at any time, will be carried out directly by you. Before acting to implement any chosen service or products, provided by or through Marquee you should consider, at your sole discretion, whether it is suitable for your particular circumstances and, if necessary, seek professional advice. Any steps necessary in order to implement the chosen service or product, including but not limited to remittance of funds, shall be carried out at your discretion. Accordingly, such services and products have not been and will not be publicly issued, placed, distributed, offered or negotiated in the Brazilian capital markets and, as a result, they have not been and will not be registered with the Brazilian Securities and Exchange Commission (Comissão de Valores Mobiliários), nor have they been submitted to the foregoing agency for approval. Documents relating to such services or products, as well as the information contained therein, may not be supplied to the general public in Brazil, as the offering of such services or products is not a public offering in Brazil, nor used in connection with any offer for subscription or sale of securities to the general public in Brazil.
The offer of any securities mentioned in this message may not be made to the general public in Brazil. Accordingly, any such securities have not been nor will they be registered with the Brazilian Securities and Exchange Commission (Comissão de Valores Mobiliários) nor has any offer been submitted to the foregoing agency for approval. Documents relating to the offer, as well as the information contained therein, may not be supplied to the public in Brazil, as the offer is not a public offering of securities in Brazil. These terms will apply on every access to Marquee.
Ouvidoria Goldman Sachs Brasil: 0800 727 5764 e/ou ouvidoriagoldmansachs@gs.com
Horário de funcionamento: segunda-feira à sexta-feira (exceto feriados), das 9hs às 18hs.
Ombudsman Goldman Sachs Brazil: 0800 727 5764 and / or ouvidoriagoldmansachs@gs.com
Available Weekdays (except holidays), from 9 am to 6 pm.
Note to Investors in Israel: GS is not licensed to provide investment advice or investment management services under Israeli law.
Notice to Investors in Japan
Marquee is made available in Japan by Goldman Sachs Japan Co., Ltd.
本書は情報の提供を目的としております。また、売却・購入が違法となるような法域での有価証券その他の売却若しくは購入を勧めるものでもありません。ゴールドマン・サックスは本書内の取引又はストラクチャーの勧誘を行うものではございません。これらの取引又はストラクチャーは、社内及び法規制等の承認等次第で実際にはご提供できない場合がございます。
<適格機関投資家限定　転売制限>
ゴールドマン・サックス証券株式会社が適格機関投資家のみを相手方として取得申込みの勧誘（取得勧誘）又は売付けの申込み若しくは買付けの申込みの勧誘(売付け勧誘等)を行う本有価証券には、適格機関投資家に譲渡する場合以外の譲渡が禁止される旨の制限が付されています。本有価証券は金融商品取引法第４条に基づく財務局に対する届出が行われておりません。なお、本告知はお客様によるご同意のもとに、電磁的に交付させていただいております。
＜適格機関投資家用資料＞
本資料は、適格機関投資家のお客さまのみを対象に作成されたものです。本資料における金融商品は適格機関投資家のお客さまのみがお取引可能であり、適格機関投資家以外のお客さまからのご注文等はお受けできませんので、ご注意ください。 商号等/ゴールドマン・サックス証券株式会社 金融商品取引業者　関東財務局長（金商）第６９号
加入協会/　日本証券業協会、一般社団法人金融先物取引業協会、一般社団法人第二種金融商品取引業協会
本書又はその添付資料に信用格付が記載されている場合、日本格付研究所（JCR）及び格付投資情報センター（R&I）による格付は、登録信用格付業者による格付（登録格付）です。その他の格付は登録格付である旨の記載がない場合は、無登録格付です。無登録格付を投資判断に利用する前に、「無登録格付に関する説明書」（http://www.goldmansachs.com/disclaimer/ratings.html）を十分にお読みください。
If any credit ratings are contained in this material or any attachments, those that have been issued by Japan Credit Rating Agency, Ltd. (JCR) or Rating and Investment Information, Inc. (R&I) are credit ratings that have been issued by a credit rating agency registered in Japan (registered credit ratings). Other credit ratings are unregistered unless denoted as being registered. Before using unregistered credit ratings to make investment decisions, please carefully read "Explanation Regarding Unregistered Credit Ratings" (http://www.goldmansachs.com/disclaimer/ratings.html).
Notice to Mexican Investors: Information contained herein is not meant for the general public in Mexico. The services or products provided by or through Goldman Sachs Mexico, Casa de Bolsa, S.A. de C.V. (GS Mexico) may not be offered or sold to the general public in Mexico. You have received information herein exclusively due to your existing relationship with a GS Mexico or any other Goldman Sachs business. The selection and engagement with any of the offered services or products through GS Mexico will be carried out directly by you at your own risk. Before acting to implement any chosen service or product provided by or through GS Mexico you should consider, at your sole discretion, whether it is suitable for your particular circumstances and, if necessary, seek professional advice. Information contained herein related to GS Mexico services or products, as well as any other information, shall not be considered as a product coming from research, nor it contains any recommendation to invest, not to invest, hold or sell any security and may not be supplied to the general public in Mexico.
Notice to New Zealand Investors: When this document is disseminated in New Zealand by Goldman Sachs & Co. LLC ("GSCO") , Goldman Sachs International ("GSI"), Goldman Sachs Bank Europe SE ("GSBE"), Goldman Sachs (Asia) L.L.C. ("GSALLC") or Goldman Sachs (Singapore) Pte ("GSSP") (collectively the "GS entities"), this document, and any access to it, is intended only for a person that has first satisfied; the GS entities that the person is someone:
(i) who is an investment business within the meaning of clause 37 of Schedule 1 of the Financial Markets Conduct Act 2013 (New Zealand) (the "FMC Act");
(ii) who meets the investment activity criteria specified in clause 38 of Schedule 1 of the FMC Act;
(iii) who is large within the meaning of clause 39 of Schedule 1 of the FMC Act; or
(iv) is a government agency within the meaning of clause 40 of Schedule 1 of the FMC Act.
No offer to acquire the interests is being made to you in this document. Any offer will only be made in circumstances where disclosure is not required under the Financial Markets Conducts Act 2013 or the Financial Markets Conduct Regulations 2014.
Notice to Swiss Investors: This is marketing material for financial instruments or services. The information contained in this material is for general informational purposes only and does not constitute an offer, solicitation, invitation or recommendation to buy or sell any financial instruments or to provide any investment advice or service of any kind.
THE INFORMATION CONTAINED IN THIS DOCUMENT DOES NOT CONSITUTE, AND IS NOT INTENDED TO CONSTITUTE, A PUBLIC OFFER OF SECURITIES IN THE UNITED ARAB EMIRATES IN ACCORDANCE WITH THE COMMERCIAL COMPANIES LAW (FEDERAL LAW NO. 2 OF 2015), ESCA BOARD OF DIRECTORS' DECISION NO. (9/R.M.) OF 2016, ESCA CHAIRMAN DECISION NO 3/R.M. OF 2017 CONCERNING PROMOTING AND INTRODUCING REGULATIONS OR OTHERWISE UNDER THE LAWS OF THE UNITED ARAB EMIRATES. ACCORDINGLY, THE INTERESTS IN THE SECURITIES MAY NOT BE OFFERED TO THE PUBLIC IN THE UAE (INCLUDING THE DUBAI INTERNATIONAL FINANCIAL CENTRE AND THE ABU DHABI GLOBAL MARKET). THIS DOCUMENT HAS NOT BEEN APPROVED BY, OR FILED WITH THE CENTRAL BANK OF THE UNITED ARAB EMIRATES, THE SECURITIES AND COMMODITIES AUTHORITY, THE DUBAI FINANCIAL SERVICES AUTHORITY, THE FINANCIAL SERVICES REGULATORY AUTHORITY OR ANY OTHER RELEVANT LICENSING AUTHORITIES IN THE UNITED ARAB EMIRATES. IF YOU DO NOT UNDERSTAND THE CONTENTS OF THIS DOCUMENT, YOU SHOULD CONSULT WITH A FINANCIAL ADVISOR. THIS DOCUMENT IS PROVIDED TO THE RECIPIENT ONLY AND SHOULD NOT BE PROVIDED TO OR RELIED ON BY ANY OTHER PERSON.